## データ整形
---

In [21]:
import pandas as pd
import os
import re
import time

### パラメータ

In [23]:
# 入力ディレクトリ
input_dir_path = "../../Study/data/detail_data_year/"
input_file_ext = "results_score.csv"

# 出力ディレクトリ
output_dir_path = "../../Study/data/"
output_file_name = "result_score_data.csv"
output_file_path = os.path.join(output_dir_path, output_file_name)

### ファイルパスのリストを取得

In [24]:
input_file_path_list = []
for root, dirs, files in os.walk(input_dir_path):
    for file in files:
        if input_file_ext in file:
            input_file_path_list.append(os.path.join(root, file))
input_file_path_list

["/Users/takayuki/Documents/work/Study/data/detail_data_year/Capital One World Men's Curling Championship 2010_results_score.csv",
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/CPT World Men’s Curling Championship 2014_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2002(Men)_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2002(Women)_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2003(Men)_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2003(Women)_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2004(Men)_results_score.csv',
 '/Users/takayuki/Documents/work/Study/data/detail_data_year/Ford World Curling Championships 2004(Women)_results_score.cs

### データの読み込み

In [25]:
start_time = time.time()

# 各試合のスコアデータを格納するDataFrame
result_score_pd = None

# game ID
game_id = 0
game_id_list = []

# 各ファイルを順に読み込み、DataFrameに格納する
for file_path in input_file_path_list:
    # CSVファイルの読み込み
    tmp_result_score_pd = pd.read_csv(file_path)
    
    # 勝敗フラグの付与（勝ち：1, 引き分け：0, 負け：-1）
    result_score_list = []
    for row_idx in range(0, tmp_result_score_pd.shape[0], 2):
        score1 = tmp_result_score_pd["score(total)"][row_idx]
        score2 = tmp_result_score_pd["score(total)"][row_idx+1]
        
        if tmp_result_score_pd["draw"][row_idx] != tmp_result_score_pd["draw"][row_idx+1]:
            print("draw is different: row %d and %d" % tmp_result_score_pd["draw"][row_idx], tmp_result_score_pd["draw"][row_idx+1])
        elif tmp_result_score_pd["sheet"][row_idx] != tmp_result_score_pd["sheet"][row_idx+1]:
            print("sheet is different: row %d and %d" % tmp_result_score_pd["draw"][row_idx], tmp_result_score_pd["draw"][row_idx+1])
        
        if score1 < score2:
            result_score_list.append(-1)
            result_score_list.append(1)
        elif score1 == score2:
            result_score_list.append(0)
            result_score_list.append(0)
        else:
            result_score_list.append(1)
            result_score_list.append(-1)
        
        # Hammerの修正(先行／後攻がわからない場合)
        hammer1 = tmp_result_score_pd["hammer"][row_idx]
        hammer2 = tmp_result_score_pd["hammer"][row_idx+1]
        if hammer1 == hammer2:
            tmp_result_score_pd["hammer"][row_idx] = -1
            tmp_result_score_pd["hammer"][row_idx+1] = -1
        
        # game ID
        game_id += 1
        game_id_list.append(game_id)
        game_id_list.append(game_id)
    
    tmp_result_score_pd["result"] = result_score_list
    
    # 年度、性別の取得
    tmp_list = re.findall(r"[0-9]+", file_path)
    if len(tmp_list) != 1 and len(tmp_list[0]) != 4:
        print("Year is missing: %d %d" % file_path, year)
    year = tmp_list[0]
    
    gender = None
    if "Women" in file_path:
        gender = "Women"
    elif "Men" in file_path:
        gender = "Men"
    else:
        print("Gender is missing: row %d." % row_idx)
    
    tmp_result_score_pd["year"] = [year] * tmp_result_score_pd.shape[0]
    tmp_result_score_pd["gender"] = [gender] * tmp_result_score_pd.shape[0]
    
    # DataFrameに追加
    if result_score_pd is None:
        result_score_pd = tmp_result_score_pd
    else:
        result_score_pd = pd.concat([result_score_pd, tmp_result_score_pd])

# game ID
result_score_pd["game_ID"] = game_id_list
        
# データの年数
year_list = list(set(result_score_pd["year"]))
year_list.sort()
print("the number of years : %d" % len(year_list))
print(year_list)

# 年×男or女
year_gender_pd = result_score_pd[["year", "gender"]]
year_gender_pd = year_gender_pd.drop_duplicates()
year_gender_pd = year_gender_pd.sort_values(by=["year", "gender"])
print("\nthe number of competition : %d" % year_gender_pd.shape[0])
year_gender_list = []
for value in year_gender_pd.values:
    year_gender_tuple = tuple(value)
    year_gender_list.append(year_gender_tuple)
print(year_gender_list)

end_time = time.time()
print("\n\tfinish file readeing: %f [sec]\n" % (end_time - start_time))

/Users/takayuki/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/takayuki/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


the number of years : 15
['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

the number of competition : 28
[('2002', 'Men'), ('2002', 'Women'), ('2003', 'Men'), ('2003', 'Women'), ('2004', 'Men'), ('2004', 'Women'), ('2005', 'Men'), ('2005', 'Women'), ('2006', 'Men'), ('2006', 'Women'), ('2007', 'Men'), ('2007', 'Women'), ('2008', 'Men'), ('2008', 'Women'), ('2009', 'Men'), ('2010', 'Men'), ('2010', 'Women'), ('2011', 'Men'), ('2012', 'Men'), ('2012', 'Women'), ('2013', 'Men'), ('2013', 'Women'), ('2014', 'Men'), ('2014', 'Women'), ('2015', 'Men'), ('2015', 'Women'), ('2016', 'Men'), ('2016', 'Women')]

	finish file readeing: 1.443008 [sec]



In [26]:
result_score_pd

,draw,sheet,team,hammer,end1,end2,end3,end4,end5,end6,...,end8,end9,end10,extra1,extra2,score(total),result,year,gender,game_ID
0,Draw #1,A,Sweden,1,0,0,0,0,0,0,...,1,0,X,NaN,NaN,1,-1,2010,Men,1
1,Draw #1,A,France,0,0,1,1,0,2,0,...,0,0,X,NaN,NaN,5,1,2010,Men,1
2,Draw #1,B,China,0,0,0,0,0,0,1,...,0,2,0,NaN,NaN,4,-1,2010,Men,2
3,Draw #1,B,Germany,1,0,0,1,1,1,0,...,1,0,1,NaN,NaN,5,1,2010,Men,2
4,Draw #1,C,Japan,0,0,1,0,1,1,0,...,0,0,1,0.0,NaN,6,-1,2010,Men,3
5,Draw #1,C,Italy,1,1,0,1,0,0,1,...,2,1,0,1.0,NaN,7,1,2010,Men,3
6,Draw #1,D,Norway,0,0,0,0,0,2,0,...,1,1,0,NaN,NaN,4,-1,2010,Men,4
7,Draw #1,D,Scotland,1,0,0,0,2,0,0,...,0,0,2,NaN,NaN,5,1,2010,Men,4
8,Draw #2,A,Italy,0,0,2,0,2,0,1,...,0,0,1,0.0,NaN,6,-1,2010,Men,5
9,Draw #2,A,China,1,1,0,2,0,0,0,...,0,1,0,1.0,NaN,7,1,2010,Men,5


### 出力

In [27]:
result_score_pd.to_csv(output_file_path, index=False, encoding="utf-8")